In [5]:
import xarray as xr
import pandas as pd

# Load your downloaded NetCDF file
ds = xr.open_dataset('E:/Arbitrage_Model_project/DATA/Wind_data_ERA5/wind_data.nc')

print(ds)

<xarray.Dataset> Size: 281kB
Dimensions:     (valid_time: 8784, latitude: 1, longitude: 1)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 70kB 2024-01-01 ... 2024-12-31T23...
  * latitude    (latitude) float64 8B 54.0
  * longitude   (longitude) float64 8B 7.0
    expver      (valid_time) <U4 141kB ...
Data variables:
    u100        (valid_time, latitude, longitude) float32 35kB ...
    v100        (valid_time, latitude, longitude) float32 35kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-11T00:05 GRIB to CDM+CF via cfgrib-0.9.1...


In [7]:
# 1. Load the CSV file:
# - header=0 reads the first row as the column names
# - skiprows=[1] skips the second junk row (at index 1)
df = pd.read_csv('E:/Arbitrage_Model_project/DATA/energy-charts_Electricity_production_and_spot_prices_in_Germany_in_2024.csv', 
                 header=0, 
                 skiprows=[1])

# 2. Rename the columns
df_prices_clean = df.rename(columns={
    'Date (GMT+1)': 'datetime',
    'Day Ahead Auction (DE-LU)': 'price_eur_mwh'
})

# 3. Convert the 'datetime' column to a real datetime object
df_prices_clean['datetime'] = pd.to_datetime(df_prices_clean['datetime'])

# 4. Set the 'datetime' as the index
df_prices_clean = df_prices_clean.set_index('datetime')

# 5. Keep only the price column
df_prices_clean = df_prices_clean[['price_eur_mwh']]

# 6. Convert the price column to a number
df_prices_clean['price_eur_mwh'] = pd.to_numeric(df_prices_clean['price_eur_mwh'])

# 7. Check your clean, ready-to-use price data!
print(df_prices_clean.head())

print("\nData information:")
df_prices_clean.info()

                           price_eur_mwh
datetime                                
2024-01-01 00:00:00+01:00           0.10
2024-01-01 01:00:00+01:00           0.01
2024-01-01 02:00:00+01:00           0.00
2024-01-01 03:00:00+01:00          -0.01
2024-01-01 04:00:00+01:00          -0.03

Data information:
<class 'pandas.core.frame.DataFrame'>
Index: 8784 entries, 2024-01-01 00:00:00+01:00 to 2024-12-31 23:00:00+01:00
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price_eur_mwh  8784 non-null   float64
dtypes: float64(1)
memory usage: 137.2+ KB


C:\Users\Madhvesh\AppData\Local\Temp\ipykernel_15208\589493000.py:15: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_prices_clean['datetime'] = pd.to_datetime(df_prices_clean['datetime'])


In [8]:
# 1. Load your downloaded NetCDF file
ds = xr.open_dataset('E:/Arbitrage_Model_project/DATA/Wind_data_ERA5/wind_data.nc')


# 2. Select the first latitude and longitude (at index 0)
#    This "flattens" the data from 3D (time, lat, lon) to 1D (time)
ds_at_point = ds.isel(latitude=0, longitude=0)

# 3. Calculate the total wind speed from the u/v components
wind_speed = (ds_at_point['u100']**2 + ds_at_point['v100']**2)**0.5

# 4. Convert this to a clean pandas DataFrame
df_wind = wind_speed.to_dataframe(name='wind_speed_ms')

# 5. NOW this will work. Convert the timezone (from UTC to CET/Berlin)
df_wind.index = df_wind.index.tz_localize('UTC').tz_convert('Europe/Berlin')

# Check your new wind DataFrame
print("--- Wind Data (Head) ---")
print(df_wind.head())


# 6. Join the two DataFrames on their datetime index
df_master = df_wind.join(df_prices_clean)


--- Wind Data (Head) ---
                           number  latitude  longitude expver  wind_speed_ms
valid_time                                                                  
2024-01-01 01:00:00+01:00       0      54.0        7.0   0001      14.317010
2024-01-01 02:00:00+01:00       0      54.0        7.0   0001      13.646270
2024-01-01 03:00:00+01:00       0      54.0        7.0   0001      13.931118
2024-01-01 04:00:00+01:00       0      54.0        7.0   0001      14.855367
2024-01-01 05:00:00+01:00       0      54.0        7.0   0001      14.652361


In [9]:
# 1. Clean up df_wind (from your output)
#    We only need the wind speed.
df_wind_clean = df_wind[['wind_speed_ms']]

# 2. Join with the price data (from your previous step)
#    (This assumes 'df_prices_clean' is still in your notebook's memory)
df_master = df_wind_clean.join(df_prices_clean)

# 3. Clean the final master DataFrame
#    This removes any hours that were in one file but not the other
#    (e.g., if one file had 8784 hours and the other 8760)
df_master = df_master.dropna()

# 4. Calculate Power Output (Same as before)
TOTAL_FARM_CAPACITY_MW = 330 

def calculate_power(wind_speed):
    if wind_speed < 4:  # Cut-in speed
        return 0.0
    elif wind_speed < 14: # Between cut-in and rated
        return (wind_speed - 4) / (14 - 4) * TOTAL_FARM_CAPACITY_MW
    elif wind_speed < 25: # Between rated and cut-out
        return TOTAL_FARM_CAPACITY_MW
    else: # Above cut-out speed
        return 0.0

# 5. Apply the function to create the new column
df_master['power_mw'] = df_master['wind_speed_ms'].apply(calculate_power)

# --- YOUR FOUNDATION IS BUILT ---

# 6. See the final, ready-to-use DataFrame
print("\n--- Final DataFrame for Modeling (Head) ---")
print(df_master.head())

# 7. Check the info to see how many hours we have
print("\n--- Final DataFrame Info (Post-Cleaning) ---")
df_master.info()


--- Final DataFrame for Modeling (Head) ---
                           wind_speed_ms  price_eur_mwh    power_mw
valid_time                                                         
2024-01-01 01:00:00+01:00      14.317010           0.01  330.000000
2024-01-01 02:00:00+01:00      13.646270           0.00  318.326903
2024-01-01 03:00:00+01:00      13.931118          -0.01  327.726894
2024-01-01 04:00:00+01:00      14.855367          -0.03  330.000000
2024-01-01 05:00:00+01:00      14.652361          -0.02  330.000000

--- Final DataFrame Info (Post-Cleaning) ---
<class 'pandas.core.frame.DataFrame'>
Index: 8783 entries, 2024-01-01 01:00:00+01:00 to 2024-12-31 23:00:00+01:00
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   wind_speed_ms  8783 non-null   float32
 1   price_eur_mwh  8783 non-null   float64
 2   power_mw       8783 non-null   float64
dtypes: float32(1), float64(2)
memory usage: 240.2+ KB


In [10]:
# 1. Calculate the hourly revenue for the "Base Case"
#    This is simply the power produced times the market price
df_master['base_revenue_eur'] = df_master['power_mw'] * df_master['price_eur_mwh']

# 2. Sum the hourly revenue to get the total for the year
total_annual_revenue_base = df_master['base_revenue_eur'].sum()

# 3. Let's see how many hours we lost money
negative_revenue_hours = df_master[df_master['base_revenue_eur'] < 0]['base_revenue_eur'].count()
total_lost_money = df_master[df_master['base_revenue_eur'] < 0]['base_revenue_eur'].sum()

# 4. Print the final results
print("--- Base Case (Wind-Only) Results ---")
print(f"Total Annual Revenue:   €{total_annual_revenue_base:,.2f}")
print(f"Hours with Negative Revenue: {negative_revenue_hours}")
print(f"Total Money Lost in Negative Hours: €{total_lost_money:,.2f}")

--- Base Case (Wind-Only) Results ---
Total Annual Revenue:   €104,543,074.47
Hours with Negative Revenue: 429
Total Money Lost in Negative Hours: €-1,265,844.11


In [11]:
# --- REAL-CASE TECHNO-ECONOMIC DATA (Siemens Silyzer 300) ---
ELECTROLYZER_MW = 50
H2_EFFICIENCY_KWH_KG = 52.2       # Real data: 17.5 MW / 335 kg/h = 52.2 kWh/kg
H2_PRICE_EUR_KG = 6.70            # Real data: Avg European Renewable H2 Cost 2024
MIN_LOAD_PCT = 0.10               # Real data: System needs 10% load (5 MW) to run

# 1. Calculate the New "Real" Breakeven Price
#    Math: (1000 kWh / 52.2 kWh/kg) * 6.70 €/kg
h2_breakeven_price = (1000 / H2_EFFICIENCY_KWH_KG) * H2_PRICE_EUR_KG
print(f"Real-Case Breakeven Price: €{h2_breakeven_price:.2f}/MWh")

# 2. Define the "Real" Arbitrage Function (with Min Load)
def calculate_real_hybrid_revenue(row):
    wind_mw = row['power_mw']
    price = row['price_eur_mwh']
    
    # Constraint: Minimum Power for Electrolyzer
    min_power_mw = ELECTROLYZER_MW * MIN_LOAD_PCT # 5 MW
    
    # LOGIC A: If prices are high, sell EVERYTHING to grid.
    # (Or if we don't have enough wind to turn on the electrolyzer)
    if (price >= h2_breakeven_price) or (wind_mw < min_power_mw):
        grid_revenue = wind_mw * price
        h2_revenue = 0
        h2_produced_kg = 0
        
    # LOGIC B: If prices are low AND we have enough wind -> Make H2
    else:
        # Send power to electrolyzer (up to max capacity)
        power_to_h2 = min(wind_mw, ELECTROLYZER_MW)
        
        # Send the REST to the grid
        power_to_grid = wind_mw - power_to_h2
        
        # Calculate H2 production using REAL efficiency
        h2_produced_kg = (power_to_h2 * 1000) / H2_EFFICIENCY_KWH_KG
        
        h2_revenue = h2_produced_kg * H2_PRICE_EUR_KG
        grid_revenue = power_to_grid * price 
        
    return pd.Series([grid_revenue, h2_revenue, h2_produced_kg])

# 3. Apply the logic
print("Running Real-Case Simulation...")
df_master[['real_grid_rev', 'real_h2_rev', 'real_h2_kg']] = df_master.apply(calculate_real_hybrid_revenue, axis=1)

# 4. Calculate Totals
df_master['total_real_revenue'] = df_master['real_grid_rev'] + df_master['real_h2_rev']
total_annual_revenue_real = df_master['total_real_revenue'].sum()
total_h2_produced_real = df_master['real_h2_kg'].sum()
revenue_uplift_real = total_annual_revenue_real - total_annual_revenue_base

print("\n--- Real-Case Solution Results ---")
print(f"Total Annual Revenue:      €{total_annual_revenue_real:,.2f}")
print(f"Revenue Uplift (Extra):    €{revenue_uplift_real:,.2f}")
print(f"Total Hydrogen Produced:   {total_h2_produced_real:,.0f} kg")

Real-Case Breakeven Price: €128.35/MWh
Running Real-Case Simulation...

--- Real-Case Solution Results ---
Total Annual Revenue:      €125,739,529.81
Revenue Uplift (Extra):    €21,196,455.34
Total Hydrogen Produced:   6,635,651 kg


In [12]:
# --- PHASE 4: TECHNO-ECONOMIC EVALUATION ---

# 1. Calculate CAPEX (Capital Expenditure)
#    Real assumption: €1,800 per kW for a full PEM system
CAPEX_PER_KW = 1800 
total_capex_eur = (ELECTROLYZER_MW * 1000) * CAPEX_PER_KW

# 2. Calculate Simple Payback Period
#    Formula: Total Cost / Annual Extra Profit
simple_payback_years = total_capex_eur / revenue_uplift_real

# 3. Calculate LCOH (Levelized Cost of Hydrogen) - Simplified
#    To be accurate, we need to include the "Opportunity Cost" of electricity
#    (The money we GAVE UP by not selling to the grid)

#    Step A: Annualized CAPEX (Mortgage payment equivalent)
#    Assume 20 year life, 7% discount rate (WACC)
WACC = 0.07
LIFETIME = 20
#    Capital Recovery Factor (CRF) formula
crf = (WACC * (1 + WACC)**LIFETIME) / ((1 + WACC)**LIFETIME - 1)
annualized_capex = total_capex_eur * crf

#    Step B: Operational Expense (OPEX)
#    Rule of thumb: 2% of CAPEX per year
annual_opex = total_capex_eur * 0.02

#    Step C: Cost of Electricity (Opportunity Cost)
#    This is the grid revenue we 'sacrificed' to make hydrogen.
#    We can calculate this by checking the grid revenue in the Base Case vs Solution Case.
#    (Note: This is a simplified proxy for LCOH calculation)
grid_revenue_lost = df_master['base_revenue_eur'].sum() - df_master['real_grid_rev'].sum()

#    Step D: Total Annual Cost
total_annual_cost = annualized_capex + annual_opex + grid_revenue_lost

#    Step E: Final LCOH
lcoh_eur_kg = total_annual_cost / total_h2_produced_real

# --- PRINT THE VERDICT ---
print("--- FINAL INVESTMENT DECISION ---")
print(f"Total CAPEX Required:      €{total_capex_eur:,.0f}")
print(f"Simple Payback Period:     {simple_payback_years:.1f} Years")
print("-" * 30)
print(f"LCOH (Production Cost):    €{lcoh_eur_kg:.2f} / kg")
print(f"Market Sale Price:         €{H2_PRICE_EUR_KG:.2f} / kg")
print(f"Profit Margin per kg:      €{H2_PRICE_EUR_KG - lcoh_eur_kg:.2f} / kg")

--- FINAL INVESTMENT DECISION ---
Total CAPEX Required:      €90,000,000
Simple Payback Period:     4.2 Years
------------------------------
LCOH (Production Cost):    €5.06 / kg
Market Sale Price:         €6.70 / kg
Profit Margin per kg:      €1.64 / kg


In [13]:
import numpy_financial as npf  # You might need to: pip install numpy-financial

# --- VERIFICATION: DISCOUNTED CASH FLOW (DCF) ANALYSIS ---

# 1. Define Assumptions
WACC = 0.07           # Weighted Average Cost of Capital (7% discount rate)
LIFETIME = 20         # Project lifespan in years
inflation_rate = 0.02 # Assume 2% inflation for OPEX/Revenue (Optional)

# 2. Create a "Cash Flow" Table
#    Year 0:  We spend the CAPEX (Negative Cash Flow)
#    Year 1-20: We earn the Revenue Uplift - OPEX (Positive Cash Flow)

cash_flows = []

# Year 0 (Initial Investment)
cash_flows.append(-total_capex_eur)

# Years 1 to 20 (Operations)
# Note: We assume revenue stays constant for simplicity (MVP approach)
annual_profit = revenue_uplift_real - annual_opex

for year in range(1, LIFETIME + 1):
    cash_flows.append(annual_profit)

# 3. Calculate Financial Metrics
#    NPV: Net Present Value
npv = npf.npv(WACC, cash_flows)

#    IRR: Internal Rate of Return
irr = npf.irr(cash_flows)

# --- PRINT THE VERDICT ---
print("--- PROFESSIONAL VERIFICATION ---")
print(f"Net Present Value (NPV):   €{npv:,.0f}")
print(f"Internal Rate of Return:   {irr:.1%}")
print("-" * 30)

if npv > 0:
    print("VERDICT: The project is VERIFIED profitable.")
    print(f"   It creates €{npv/1000000:.1f} Million in value today.")
else:
    print("VERDICT: The project destroys value (NPV is negative).")

--- PROFESSIONAL VERIFICATION ---
Net Present Value (NPV):   €115,486,324
Internal Rate of Return:   21.1%
------------------------------
VERDICT: The project is VERIFIED profitable.
   It creates €115.5 Million in value today.
